In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import sys
from sec_edgar_downloader import Downloader
from tqdm import tqdm

In [2]:
# Extract negtive words list from H4N
H4N = []

file = open("/Users/juntao/project_5/Harvard IV_Negative Word List_Inf.txt", 'r')
line = file.readline()
while line:
    H4N.append(line)
    line = file.readline()
file.close()

for i in range(len(H4N)):
    H4N[i] = H4N[i].lower() # to lowercase
    H4N[i] = H4N[i].rstrip('\n') # delete space

def convert_list(lst):
    dic = {i:0 for i in lst}
    return dic

H4N = convert_list(H4N)


In [3]:
# Extract negtive words list from Fin-Neg
Fin_Neg = []

import pandas as pd
import numpy as np

Fin_Neg_Dic = pd.read_csv("/Users/juntao/project_5/Loughran-McDonald_MasterDictionary_1993-2021.csv")

for i in range(len(Fin_Neg_Dic.index)):
    if Fin_Neg_Dic.Negative[i] != 0:
        Fin_Neg.append(Fin_Neg_Dic.Word[i])

for i in range(len(Fin_Neg)):
    Fin_Neg[i] = Fin_Neg[i].lower() # to lowercase
    
Fin_Neg = convert_list(Fin_Neg)

In [4]:
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#remove stopwords
### Functions for various textual cleaning tasks: 
stop_words = set(stopwords.words('english'))

# convert all text to lower case and remove non alphabetic components
def text_clean(raw_text):
    # 1. convert to lower case
    txt1 = raw_text.lower().strip()
    
    # 2. remove non-letters 
    txt2 = re.sub("[^a-zA-Z]", " ", txt1).split(' ')
    return (txt2)


# convert sentences to list
def sent_to_words(sentences):
    for sent in sentences:
        yield(text_clean(sent))

        
# Remove Stop Words
def remove_stopwords(texts):
    nostop = [word for word in texts if word not in stop_words]
    return nostop

In [5]:
tickers = pd.read_csv('/Users/juntao/project_5/tickers.csv')

In [6]:
tickers = tickers.set_index('Unnamed: 0')

In [7]:
df = pd.DataFrame(columns = ["name", "cik", "ticker",
                             "H4N_ratio", "Fin_Neg_ratio",
                             "H4N_tfidf", "Fin_Neg_tfidf",
                             "text", "filling_date", "return"])

In [8]:
from bs4 import BeautifulSoup
import os
k = 0
for folder in os.listdir('/Users/juntao/project_5/sec-edgar-filings'):
    if folder == ".DS_Store" :
        continue #folder is ticker name
    for subFolder in os.listdir('/Users/juntao/project_5/sec-edgar-filings/' + folder):
        if subFolder == ".DS_Store":
            continue #subFolder is 10-k or 10-q
        for sub_subFolder in os.listdir('/Users/juntao/project_5/sec-edgar-filings/' + folder + '/'+ subFolder):
            if sub_subFolder == ".DS_Store":
                continue
            path = '/Users/juntao/project_5/sec-edgar-filings/' + folder +'/' + subFolder +'/'+ sub_subFolder +'/'+ 'filing-details.html'
            text_file = open(path, "r", encoding="utf8")
            soup = BeautifulSoup(text_file.read()).get_text()
            text_file.close()
            
            text_nonum = text_clean(soup)
            soup = remove_stopwords(text_nonum)
            
            n = 0
            for i in soup:
                if i in H4N:
                    n = n+1
            m = 0
            for i in soup:
                if i in Fin_Neg:
                    m = m+1
                    
            H4N_ratio = n / len(soup)
            Fin_Neg_ratio = m / len(soup)
        
            df.loc[k] = {'name': sub_subFolder,
                     'H4N_ratio': H4N_ratio,
                     'Fin_Neg_ratio': Fin_Neg_ratio,
                     'text': soup}
            print(path, 'success')
            k = k + 1


/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-20-000003/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-21-000009/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-21-000002/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-21-000032/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-20-000041/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-Q/0000723254-20-000014/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-K/0000723254-21-000021/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTAS/10-K/0000723254-20-000025/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/VZ/10-Q/0000732712-20-000027/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/VZ/10-Q/0000732712-21-000024/filing-details.html su

/Users/juntao/project_5/sec-edgar-filings/KHC/10-Q/0001637459-20-000061/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KHC/10-Q/0001637459-21-000047/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KHC/10-Q/0001637459-20-000131/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KHC/10-Q/0001637459-20-000159/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KHC/10-K/0001637459-21-000009/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KHC/10-K/0001637459-20-000027/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/TEL/10-Q/0001558370-20-000310/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/TEL/10-Q/0001558370-20-005005/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/TEL/10-Q/0001558370-21-004696/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/TEL/10-Q/0001558370-21-009561/filing-details.html success


/Users/juntao/project_5/sec-edgar-filings/FBHS/10-Q/0001564590-20-048855/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FBHS/10-Q/0001564590-20-034912/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FBHS/10-Q/0001564590-21-052308/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FBHS/10-Q/0001564590-21-039114/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FBHS/10-K/0001564590-20-006812/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FBHS/10-K/0001564590-21-008129/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PFE/10-Q/0000078003-21-000065/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PFE/10-Q/0000078003-20-000060/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PFE/10-Q/0000078003-20-000037/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PFE/10-Q/0000078003-21-000093/filing-details.html su

/Users/juntao/project_5/sec-edgar-filings/ABC/10-Q/0001140859-21-000045/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABC/10-Q/0001140859-20-000022/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABC/10-K/0001140859-20-000050/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABC/10-K/0001140859-21-000058/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-21-000079/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-20-000067/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-20-000041/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-21-000106/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-21-000032/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/KMI/10-Q/0001506307-20-000097/filing-details.html success


/Users/juntao/project_5/sec-edgar-filings/SNPS/10-Q/0000883241-20-000009/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/SNPS/10-Q/0000883241-20-000006/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/SNPS/10-Q/0000883241-20-000012/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/SNPS/10-Q/0000883241-21-000017/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/SNPS/10-K/0000883241-21-000022/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/SNPS/10-K/0000883241-20-000015/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PLD/10-Q/0001564590-20-020301/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PLD/10-Q/0001564590-21-038308/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PLD/10-Q/0001564590-21-020897/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PLD/10-Q/0001564590-21-052041/filing-details.html su

...

/Users/juntao/project_5/sec-edgar-filings/CTLT/10-Q/0001596783-20-000224/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTLT/10-Q/0001596783-20-000017/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTLT/10-Q/0001596783-21-000183/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTLT/10-Q/0001596783-21-000061/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTLT/10-K/0001596783-21-000110/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/CTLT/10-K/0001596783-20-000155/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABBV/10-Q/0001551152-21-000016/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABBV/10-Q/0001551152-20-000023/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABBV/10-Q/0001551152-20-000015/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/ABBV/10-Q/0001551152-21-000031/filing-details.htm

/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-21-000041/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-21-000055/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-20-000041/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-20-000025/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-21-000016/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-Q/0001645590-20-000009/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-K/0001645590-20-000056/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/HPE/10-K/0001645590-21-000068/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/EFX/10-Q/0000033185-21-000044/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/EFX/10-Q/0000033185-21-000032/filing-details.html success


/Users/juntao/project_5/sec-edgar-filings/PNR/10-Q/0000077360-21-000035/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PNR/10-Q/0000077360-21-000025/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PNR/10-Q/0000077360-20-000018/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PNR/10-Q/0000077360-21-000012/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PNR/10-K/0000077360-21-000005/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PNR/10-K/0000077360-20-000006/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FISV/10-Q/0000798354-20-000020/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FISV/10-Q/0000798354-21-000014/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FISV/10-Q/0000798354-21-000008/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/FISV/10-Q/0000798354-21-000018/filing-details.html succ

/Users/juntao/project_5/sec-edgar-filings/PRGO/10-Q/0001585364-20-000049/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PRGO/10-Q/0001585364-20-000083/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PRGO/10-K/0001585364-21-000012/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/PRGO/10-K/0001585364-20-000013/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-21-000058/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-20-000150/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-21-000136/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-20-000081/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-21-000105/filing-details.html success
/Users/juntao/project_5/sec-edgar-filings/LYV/10-Q/0001335258-20-000163/filing-details.html succ

FileNotFoundError: [Errno 2] No such file or directory: '/Users/juntao/project_5/sec-edgar-filings/HSY/10-K/0000047111-20-000007/filing-details.html'

In [9]:
df.to_csv('/Users/juntao/project_5/Analysis_text.csv')

In [10]:
df.head()

,name,cik,ticker,H4N_ratio,Fin_Neg_ratio,H4N_tfidf,Fin_Neg_tfidf,text,filling_date,return
0,0000723254-20-000003,NaN,NaN,0.014070,0.003447,NaN,NaN,"[document, , , false, , , , , , , q, , , , , ,...",NaN,NaN
1,0000723254-21-000009,NaN,NaN,0.019208,0.005929,NaN,NaN,"[ctas, , , , , , , , , , , , , , , , , , , fal...",NaN,NaN
2,0000723254-21-000002,NaN,NaN,0.021302,0.006451,NaN,NaN,"[ctas, , , , , , , , , , , , , , , , , , , fal...",NaN,NaN
3,0000723254-21-000032,NaN,NaN,0.019405,0.004288,NaN,NaN,"[ctas, , , , , , , , , , , , , , , , , , , fal...",NaN,NaN
4,0000723254-20-000041,NaN,NaN,0.022042,0.007275,NaN,NaN,"[ctas, , , , , , , , , , , , , , , , , , , fal...",NaN,NaN


In [27]:
df = pd.read_csv('/Users/juntao/project_5/Analysis_text.csv')

In [18]:
df.head()

,Unnamed: 0,name,cik,ticker,H4N_ratio,Fin_Neg_ratio,H4N_tfidf,Fin_Neg_tfidf,text,filling_date,return
0,0,0000723254-20-000003,NaN,NaN,0.014070,0.003447,NaN,NaN,"['document', '', '', 'false', '', '', '', '', ...",NaN,NaN
1,1,0000723254-21-000009,NaN,NaN,0.019208,0.005929,NaN,NaN,"['ctas', '', '', '', '', '', '', '', '', '', '...",NaN,NaN
2,2,0000723254-21-000002,NaN,NaN,0.021302,0.006451,NaN,NaN,"['ctas', '', '', '', '', '', '', '', '', '', '...",NaN,NaN
3,3,0000723254-21-000032,NaN,NaN,0.019405,0.004288,NaN,NaN,"['ctas', '', '', '', '', '', '', '', '', '', '...",NaN,NaN
4,4,0000723254-20-000041,NaN,NaN,0.022042,0.007275,NaN,NaN,"['ctas', '', '', '', '', '', '', '', '', '', '...",NaN,NaN


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1917 non-null   int64  
 1   name           1917 non-null   object 
 2   cik            0 non-null      float64
 3   ticker         0 non-null      float64
 4   H4N_ratio      1917 non-null   float64
 5   Fin_Neg_ratio  1917 non-null   float64
 6   H4N_tfidf      0 non-null      float64
 7   Fin_Neg_tfidf  0 non-null      float64
 8   text           1917 non-null   object 
 9   filling_date   0 non-null      float64
 10  return         0 non-null      float64
dtypes: float64(8), int64(1), object(2)
memory usage: 164.9+ KB
